In [2]:
import os
from PIL import Image
from torch.utils.data import Dataset
import torch
import pandas as pd
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim


In [3]:
class ImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.is_test = is_test
        self.class_mapping = {'AK': 0, 'BCC': 1, 'BKL': 2, 'DF': 3, 'MEL': 4, 
                              'NV': 5, 'SCC': 6, 'VASC': 7}


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if not self.is_test:
            
            img_path = os.path.join(self.root_dir, self.data.iloc[idx, 1])
            image = Image.open(img_path).convert('RGB')
            class_name = self.data.iloc[idx, 0]  
            label = self.class_mapping.get(class_name, -1)  
            label = torch.tensor(label, dtype=torch.long)

            if self.transform:
                image = self.transform(image)

            return image, label
        else:
            # For test data
            img_path = os.path.join(self.root_dir, self.data.iloc[idx, 4])
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, self.data.iloc[idx, 0]

In [4]:
image_size = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),  # ⚠️ Di chuyển ToTensor() lên trước
    transforms.Normalize(mean, std),
    transforms.GaussianBlur(kernel_size=3),
    transforms.RandomErasing(p=0.7)
])

valid_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [5]:
train_dataset = ImageDataset(
    csv_file='/kaggle/input/skincancer-isic2019/image_paths.csv',  
    root_dir='/kaggle/input/skincancer-isic2019/archive/archive',
    transform=train_transform
)

test_dataset = ImageDataset(
    csv_file='/kaggle/input/model-datatest-skincancer/test_data_remove_unk.csv', 
    root_dir='/kaggle/input/isic-2019-challenge/ISIC_2019_Test_Input/ISIC_2019_Test_Input', 
    transform=valid_transform, is_test=True)

In [6]:
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size

trainset, validset = random_split(train_dataset, [train_size, valid_size])


batch_size = 32
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(validset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes=8):
        super(CustomEfficientNet, self).__init__()

        # Load EfficientNet-B0 pre-trained model
        self.base_model = models.efficientnet_b1(pretrained=True)
        
        # Modify the final fully connected layer
        in_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.base_model(x)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CustomEfficientNet(num_classes=8).to(device)

optimizer = optim.RMSprop(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B1_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-bac287d4.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1_rwightman-bac287d4.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 170MB/s]


In [56]:
# def train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25):
#     best_val_acc = 0.0
#     for epoch in range(num_epochs):
#         model.train()  # Set to training mode
#         running_loss, running_corrects = 0.0, 0

#         # Training loop
#         for inputs, labels in train_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

#             _, preds = torch.max(outputs, 1)
#             running_loss += loss.item() * inputs.size(0)
#             running_corrects += torch.sum(preds == labels.data)

#         epoch_loss = running_loss / len(train_loader.dataset)
#         epoch_acc = running_corrects.double() / len(train_loader.dataset)
#         print(f'Training - Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

#         # Validation loop
#         model.eval()  # Set to evaluation mode
#         val_corrects = 0
#         with torch.no_grad():
#             for inputs, labels in val_loader:
#                 inputs, labels = inputs.to(device), labels.to(device)
#                 outputs = model(inputs)
#                 _, preds = torch.max(outputs, 1)
#                 val_corrects += torch.sum(preds == labels.data)

#         val_acc = val_corrects.double() / len(val_loader.dataset)
#         print(f'Validation Accuracy: {val_acc:.4f}')

#         # Save the best model based on validation accuracy
#         if val_acc > best_val_acc:
#             best_val_acc = val_acc
#             torch.save(model.state_dict(), 'best_model.pth')

In [65]:
def train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=25):
    best_val_acc = 0.0
    history = {
        "train_loss": [],
        "val_loss": [],
        "train_acc": [],
        "val_acc": []
    }

    for epoch in range(num_epochs):
        model.train()  
        running_loss, running_corrects = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        train_epoch_loss = running_loss / len(train_loader.dataset)
        train_epoch_acc = running_corrects.double() / len(train_loader.dataset)

        
        model.eval()  
        val_loss, val_corrects = 0.0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                _, preds = torch.max(outputs, 1)
                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_epoch_loss = val_loss / len(val_loader.dataset)
        val_epoch_acc = val_corrects.double() / len(val_loader.dataset)

      
        history["train_loss"].append(train_epoch_loss)
        history["val_loss"].append(val_epoch_loss)
        history["train_acc"].append(train_epoch_acc.item())
        history["val_acc"].append(val_epoch_acc.item())

        print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {train_epoch_loss:.4f}, Train Acc: {train_epoch_acc:.4f} - Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}')

        
        if val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            torch.save(model.state_dict(), 'best_model.pth')

    return history

In [57]:
train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=15)

Training - Epoch 1/15, Loss: 0.9996, Accuracy: 0.6491
Validation Accuracy: 0.6988
Training - Epoch 2/15, Loss: 0.8225, Accuracy: 0.7075
Validation Accuracy: 0.7186
Training - Epoch 3/15, Loss: 0.7403, Accuracy: 0.7352
Validation Accuracy: 0.7251
Training - Epoch 4/15, Loss: 0.6812, Accuracy: 0.7566
Validation Accuracy: 0.7630
Training - Epoch 5/15, Loss: 0.6305, Accuracy: 0.7753
Validation Accuracy: 0.7665
Training - Epoch 6/15, Loss: 0.5905, Accuracy: 0.7884
Validation Accuracy: 0.7778
Training - Epoch 7/15, Loss: 0.5456, Accuracy: 0.8031
Validation Accuracy: 0.7657
Training - Epoch 8/15, Loss: 0.5086, Accuracy: 0.8180
Validation Accuracy: 0.7878
Training - Epoch 9/15, Loss: 0.4845, Accuracy: 0.8242
Validation Accuracy: 0.7861
Training - Epoch 10/15, Loss: 0.4527, Accuracy: 0.8384
Validation Accuracy: 0.7930
Training - Epoch 11/15, Loss: 0.4303, Accuracy: 0.8455
Validation Accuracy: 0.8011
Training - Epoch 12/15, Loss: 0.4032, Accuracy: 0.8561
Validation Accuracy: 0.8038
Training - Ep

In [66]:
for param in model.base_model.features[6:].parameters():
    param.requires_grad = True

# Re-define optimizer for fine-tuning
optimizer = optim.SGD(model.parameters(), lr=0.0001)

# Fine-tune the model
history = train_and_val_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

Epoch 1/10 - Train Loss: 0.3167, Train Acc: 0.8878 - Val Loss: 0.6010, Val Acc: 0.8084
Epoch 2/10 - Train Loss: 0.3137, Train Acc: 0.8877 - Val Loss: 0.5714, Val Acc: 0.8133
Epoch 3/10 - Train Loss: 0.3068, Train Acc: 0.8875 - Val Loss: 0.5825, Val Acc: 0.8180
Epoch 4/10 - Train Loss: 0.2998, Train Acc: 0.8923 - Val Loss: 0.5728, Val Acc: 0.8174
Epoch 5/10 - Train Loss: 0.3043, Train Acc: 0.8908 - Val Loss: 0.5684, Val Acc: 0.8186
Epoch 6/10 - Train Loss: 0.2988, Train Acc: 0.8918 - Val Loss: 0.5889, Val Acc: 0.8131
Epoch 7/10 - Train Loss: 0.3020, Train Acc: 0.8938 - Val Loss: 0.5826, Val Acc: 0.8171
Epoch 8/10 - Train Loss: 0.2947, Train Acc: 0.8935 - Val Loss: 0.5821, Val Acc: 0.8171
Epoch 9/10 - Train Loss: 0.2991, Train Acc: 0.8942 - Val Loss: 0.5784, Val Acc: 0.8167
Epoch 10/10 - Train Loss: 0.2988, Train Acc: 0.8898 - Val Loss: 0.5576, Val Acc: 0.8228


In [69]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history["train_loss"], label="Train Loss", marker='o')
plt.plot(history["val_loss"], label="Validation Loss", marker='o')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Train & Validation Loss")
plt.legend()

# Vẽ biểu đồ Train Accuracy và Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(history["train_acc"], label="Train Accuracy", marker='o')
plt.plot(history["val_acc"], label="Validation Accuracy", marker='o')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Train & Validation Accuracy")
plt.legend()

plt.show()

In [9]:
model.load_state_dict(torch.load('/kaggle/input/effbo_2502/pytorch/default/1/effb0_2502.pth', map_location=device))


model.eval()

<ipython-input-9-4deb0a6f41e6>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/input/effbo_2502/pytorch/default/1/effb0_2502.pth',

CustomEfficientNet(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
            

In [10]:
def predict(model, test_loader):
    model.eval()  
    predictions = []
    image_ids = []

    with torch.no_grad(): 
        for images, img_id in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)  
            
            predictions.extend(preds.cpu().numpy()) 
            image_ids.extend(img_id)  

    return image_ids, predictions

In [11]:
image_ids, preds = predict(model, test_loader)
class_mapping_inv = {v: k for k, v in train_dataset.class_mapping.items()}
pred_classes = [class_mapping_inv[p] for p in preds]

In [12]:
submission_df = pd.DataFrame({'image_id': image_ids, 'predicted_class': pred_classes})

In [13]:
submission_df

,image_id,predicted_class
0,ISIC_0034321,BKL
1,ISIC_0034322,NV
2,ISIC_0034323,BCC
3,ISIC_0034324,NV
4,ISIC_0034325,NV
...,...,...
6186,ISIC_0073226,BCC
6187,ISIC_0073234,NV
6188,ISIC_0073236,BCC
6189,ISIC_0073243,BCC


In [14]:
test_gt = pd.read_csv('/kaggle/input/model-datatest-skincancer/test_gt_remove_unk.csv')
test_gt.head()

,image,diagnosis
0,ISIC_0034321,NV
1,ISIC_0034322,NV
2,ISIC_0034323,BCC
3,ISIC_0034324,NV
4,ISIC_0034325,NV


In [15]:
accuracy = (submission_df['predicted_class'] == test_gt['diagnosis']).mean()
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.7068


# Evaluation

In [16]:
test_gt.rename(columns={"image": "image_id", "diagnosis": "true_class"}, inplace=True)

df_merged = submission_df.merge(test_gt, on="image_id")

misclassified_df = df_merged[df_merged['predicted_class'] != df_merged['true_class']]
print(len(misclassified_df))
display(misclassified_df.head())


1815


,image_id,predicted_class,true_class
0,ISIC_0034321,BKL,NV
7,ISIC_0034329,NV,MEL
21,ISIC_0034343,NV,MEL
24,ISIC_0034346,SCC,BCC
31,ISIC_0034354,BKL,MEL


In [18]:
import torch.nn.functional as F
def predict_with_prob(model, test_loader):
    model.eval()
    predictions = []
    image_ids = []
    probabilities = []

    with torch.no_grad():
        for images, img_id in test_loader:
            images = images.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)  # Convert logits thành xác suất
            max_probs, preds = torch.max(probs, 1)  

            predictions.extend(preds.cpu().numpy())
            image_ids.extend(img_id)
            probabilities.extend(max_probs.cpu().numpy())  # Lưu xác suất

    return image_ids, predictions, probabilities


image_ids, preds, probs = predict_with_prob(model, test_loader)


submission_df = pd.DataFrame({'image_id': image_ids, 'predicted_class': pred_classes, 'probability': probs})

In [19]:
submission_df

,image_id,predicted_class,probability
0,ISIC_0034321,BKL,0.732439
1,ISIC_0034322,NV,0.999998
2,ISIC_0034323,BCC,0.999567
3,ISIC_0034324,NV,1.000000
4,ISIC_0034325,NV,1.000000
...,...,...,...
6186,ISIC_0073226,BCC,0.986773
6187,ISIC_0073234,NV,0.918639
6188,ISIC_0073236,BCC,0.961431
6189,ISIC_0073243,BCC,0.625059


In [20]:
uncertain_df = submission_df[(submission_df['probability'] >= 0.45) & (submission_df['probability'] <= 0.55)]
print(len(uncertain_df))
display(uncertain_df.head())

383


,image_id,predicted_class,probability
95,ISIC_0034422,BCC,0.456413
104,ISIC_0034431,NV,0.537871
131,ISIC_0034459,BKL,0.540407
216,ISIC_0034547,BKL,0.508060
242,ISIC_0034574,BCC,0.488606
